In [ ]:
import numpy as np

def kc_penalty(w, gamma=1e-2, eta=1e-2):
    """Approximate Kolmogorov complexity penalty as a combination of L0 and L1 penalties."""
    l0 = np.count_nonzero(w)
    l1 = np.sum(np.abs(w))
    return gamma * l0 + eta * l1

def objective(w, Sigma, mu, lam, gamma, eta):
    """Full objective function: risk, return, and KC penalty."""
    risk = w.T @ Sigma @ w
    ret = w.T @ mu
    penalty = kc_penalty(w, gamma, eta)
    return risk - lam * ret + penalty

def grad_objective(w, Sigma, mu, lam, eta):
    """Gradient of the objective function."""
    grad_risk = 2 * Sigma @ w
    grad_ret = lam * mu
    grad_l1 = eta * np.sign(w)
    return grad_risk - grad_ret + grad_l1

def optimize_weights(Sigma, mu, lam=0.1, gamma=1e-2, eta=1e-2, lr=1e-2, epochs=500, verbose=True):
    """Main optimization loop with verbose output and KC penalty regularization."""
    n_assets = len(mu)
    w = np.ones(n_assets) / n_assets  # Start with equal weights

    log_lines = []
    log_lines.append("==== Kolmogorov Complexity Constraint Portfolio Optimization ====\n")
    log_lines.append(f"Initial weights: {w}\n")
    log_lines.append(f"lambda: {lam}, gamma: {gamma}, eta: {eta}, lr: {lr}, epochs: {epochs}\n\n")

    for epoch in range(1, epochs + 1):
        grad = grad_objective(w, Sigma, mu, lam, eta)
        w -= lr * grad

        # Apply approximate L0 penalty (threshold small values to zero)
        w[np.abs(w) < gamma] = 0

        # Optional projection to ensure weights are non-negative and sum to one
        w = np.maximum(w, 0)
        if np.sum(w) > 0:
            w /= np.sum(w)

        # Calculate current objective value
        obj_val = objective(w, Sigma, mu, lam, gamma, eta)
        penalty_val = kc_penalty(w, gamma, eta)
        sparsity = np.count_nonzero(w)

        if verbose:
            print(f"Epoch {epoch}/{epochs}")
            print(f"  Weights: {w}")
            print(f"  Objective value: {obj_val:.6f}")
            print(f"  Penalty value: {penalty_val:.6f}")
            print(f"  Sparsity (non-zero weights): {sparsity}\n")

        log_lines.append(f"Epoch {epoch}: Weights: {w.tolist()}, Objective: {obj_val:.6f}, Penalty: {penalty_val:.6f}, Sparsity: {sparsity}\n")

    log_lines.append("\n==== Optimization Completed ====\n")
    log_lines.append(f"Final weights: {w}\n")
    log_lines.append(f"Final sparsity (non-zero): {np.count_nonzero(w)}\n")
    log_lines.append("============================================================\n")

    # Write log to text file
    with open("kc_portfolio_verbose_log.txt", "w") as f:
        f.writelines(log_lines)

    print("✅ Log saved to kc_portfolio_verbose_log.txt")

    return w

# Example usage
if __name__ == "__main__":
    np.random.seed(42)

    n_assets = 5
    Sigma = np.random.rand(n_assets, n_assets)
    Sigma = Sigma @ Sigma.T  # Make it symmetric and positive semi-definite
    mu = np.random.rand(n_assets)

    optimized_weights = optimize_weights(
        Sigma,
        mu,
        lam=0.5,
        gamma=0.01,
        eta=0.01,
        lr=0.01,
        epochs=100,
        verbose=True
    )

    print("\n✅ Final optimized weights:", optimized_weights)


Epoch 1/100
  Weights: [0.1952489  0.19654665 0.19778107 0.20577626 0.20464711]
  Objective value: 0.890859
  Penalty value: 0.060000
  Sparsity (non-zero weights): 5

Epoch 2/100
  Weights: [0.19005822 0.19271724 0.19539829 0.21212774 0.2096985 ]
  Objective value: 0.881640
  Penalty value: 0.060000
  Sparsity (non-zero weights): 5

Epoch 3/100
  Weights: [0.184393   0.18847639 0.19284288 0.21910438 0.21518335]
  Objective value: 0.871661
  Penalty value: 0.060000
  Sparsity (non-zero weights): 5

Epoch 4/100
  Weights: [0.17821669 0.18378647 0.19010619 0.22675891 0.22113174]
  Objective value: 0.860881
  Penalty value: 0.060000
  Sparsity (non-zero weights): 5

Epoch 5/100
  Weights: [0.17149138 0.17860766 0.1871799  0.23514666 0.22757441]
  Objective value: 0.849261
  Penalty value: 0.060000
  Sparsity (non-zero weights): 5

Epoch 6/100
  Weights: [0.16417802 0.17289811 0.1840561  0.2443253  0.23454247]
  Objective value: 0.836764
  Penalty value: 0.060000
  Sparsity (non-zero weigh

In [ ]:
import numpy as np

# Helper functions (reuse from main code)
def kc_penalty(w, gamma=1e-2, eta=1e-2):
    l0 = np.count_nonzero(w)
    l1 = np.sum(np.abs(w))
    return gamma * l0 + eta * l1

def objective(w, Sigma, mu, lam, gamma, eta):
    risk = w.T @ Sigma @ w
    ret = w.T @ mu
    penalty = kc_penalty(w, gamma, eta)
    return risk - lam * ret + penalty

def grad_objective(w, Sigma, mu, lam, eta):
    grad_risk = 2 * Sigma @ w
    grad_ret = lam * mu
    grad_l1 = eta * np.sign(w)
    return grad_risk - grad_ret + grad_l1

def optimize_weights(Sigma, mu, lam, gamma, eta, lr=1e-2, epochs=300, verbose=False):
    n_assets = len(mu)
    w = np.ones(n_assets) / n_assets

    for epoch in range(epochs):
        grad = grad_objective(w, Sigma, mu, lam, eta)
        w -= lr * grad
        w[np.abs(w) < gamma] = 0
        w = np.maximum(w, 0)
        if np.sum(w) > 0:
            w /= np.sum(w)
    return w

# Create one large log string
master_log = ""

# Hyperparameter ranges
gammas = [0.001, 0.01, 0.05, 0.1]
etas = [0.001, 0.01, 0.05, 0.1]
lambdas = [0.1, 0.5, 1.0, 5.0]

# Number of assets
n_assets_list = [5, 20, 50, 100]

# Loop over all configurations
for n_assets in n_assets_list:
    np.random.seed(42)
    Sigma = np.random.rand(n_assets, n_assets)
    Sigma = Sigma @ Sigma.T + 1e-3 * np.eye(n_assets)  # ensure positive definiteness
    mu = np.random.rand(n_assets)

    for gamma in gammas:
        for eta in etas:
            for lam in lambdas:
                weights = optimize_weights(Sigma, mu, lam, gamma, eta, lr=1e-2, epochs=300, verbose=False)
                sparsity = np.count_nonzero(weights)
                final_obj = objective(weights, Sigma, mu, lam, gamma, eta)
                penalty_val = kc_penalty(weights, gamma, eta)

                # Append to master log
                master_log += "==== Kolmogorov Complexity Stress Test ====\n"
                master_log += f"Number of assets: {n_assets}\n"
                master_log += f"Gamma (L0 penalty): {gamma}\n"
                master_log += f"Eta (L1 penalty): {eta}\n"
                master_log += f"Lambda (risk-return trade-off): {lam}\n\n"
                master_log += f"Final weights: {weights.tolist()}\n"
                master_log += f"Sparsity (non-zero): {sparsity}\n"
                master_log += f"Final objective value: {final_obj:.6f}\n"
                master_log += f"Penalty value: {penalty_val:.6f}\n"
                master_log += "===========================================\n\n"

# Save master log to single file
with open("kc_test_master_log.txt", "w") as f:
    f.write(master_log)

print("✅ All large-scale stress tests completed.")
print("✅ Master log saved to kc_test_master_log.txt")

✅ All large-scale stress tests completed.
✅ Master log saved to kc_test_master_log.txt


In [4]:
import numpy as np

def kc_penalty(w, gamma=1e-2, eta=1e-2):
    return gamma * np.count_nonzero(w) + eta * np.sum(np.abs(w))

def objective(w, Sigma, mu, lam, gamma, eta):
    risk = w.T @ Sigma @ w
    ret = w.T @ mu
    return risk - lam * ret + kc_penalty(w, gamma, eta)

def grad_objective(w, Sigma, mu, lam, eta):
    return 2 * Sigma @ w - lam * mu + eta * np.sign(w)

def optimize_weights(Sigma, mu, lam, gamma, eta, lr=1e-2, epochs=300):
    w = np.ones_like(mu) / len(mu)
    for _ in range(epochs):
        grad = grad_objective(w, Sigma, mu, lam, eta)
        w -= lr * grad
        w[np.abs(w) < gamma] = 0
        w = np.maximum(w, 0)
        if np.sum(w) > 0:
            w /= np.sum(w)
    return w

# Base setup
np.random.seed(42)
n_assets = 20
Sigma = np.random.rand(n_assets, n_assets)
Sigma = Sigma @ Sigma.T + 1e-3 * np.eye(n_assets)
mu = np.random.rand(n_assets)

# Original
w_base = optimize_weights(Sigma, mu, lam=1.0, gamma=0.01, eta=0.01)

# Perturbations
epsilon = 1e-3
Sigma_pert = Sigma + epsilon * np.random.randn(n_assets, n_assets)
Sigma_pert = Sigma_pert @ Sigma_pert.T + 1e-3 * np.eye(n_assets)
mu_pert = mu + epsilon * np.random.randn(n_assets)

w_pert = optimize_weights(Sigma_pert, mu_pert, lam=1.0, gamma=0.01, eta=0.01)

print("Original weights:", w_base)
print("Perturbed weights:", w_pert)
print("Difference (L2 norm):", np.linalg.norm(w_base - w_pert))


Original weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Perturbed weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Difference (L2 norm): 0.0


In [5]:
# Reuse functions above

np.random.seed(42)
n_assets = 20
Sigma = np.random.rand(n_assets, n_assets)
Sigma = Sigma @ Sigma.T + 1e-3 * np.eye(n_assets)
mu = np.random.rand(n_assets)

# Extremely high penalties
gamma_high = 5.0
eta_high = 5.0

w_collapse = optimize_weights(Sigma, mu, lam=1.0, gamma=gamma_high, eta=eta_high)

print("Extreme penalty weights:", w_collapse)
print("Sparsity (non-zero count):", np.count_nonzero(w_collapse))


Extreme penalty weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Sparsity (non-zero count): 0


In [6]:
# Reuse functions above

np.random.seed(42)
n_assets = 20
Sigma = np.random.rand(n_assets, n_assets)
Sigma = Sigma @ Sigma.T + 1e-3 * np.eye(n_assets)
mu = np.random.rand(n_assets)

results = []

for trial in range(5):
    # Random initialization instead of uniform
    w_init = np.random.rand(n_assets)
    w_init /= np.sum(w_init)

    def optimize_from_custom(w_start):
        w = w_start.copy()
        for _ in range(300):
            grad = grad_objective(w, Sigma, mu, lam=1.0, eta=0.01)
            w -= 1e-2 * grad
            w[np.abs(w) < 0.01] = 0
            w = np.maximum(w, 0)
            if np.sum(w) > 0:
                w /= np.sum(w)
        return w

    w_result = optimize_from_custom(w_init)
    results.append(w_result)
    print(f"Trial {trial + 1} final weights:", w_result)
    print("Sparsity:", np.count_nonzero(w_result))
    print("=====")

# Compare if solutions converge to same sparse pattern


Trial 1 final weights: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Sparsity: 1
=====
Trial 2 final weights: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Sparsity: 1
=====
Trial 3 final weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Sparsity: 0
=====
Trial 4 final weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Sparsity: 0
=====
Trial 5 final weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Sparsity: 0
=====
